In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from numpy import array

import os
import joblib
import pickle

import random
import datetime
from pymongo import MongoClient

from sklearn import model_selection
from lightgbm import LGBMClassifier

import warnings 
warnings.filterwarnings(action='ignore')

def get_mongo_data(db_name,collection_name,condition = False):
    server_address = "private_server_address"
    client_update = MongoClient("mongodb://private_company_client/admin".format(server_address))
    db = client_update[db_name]
    col = db[collection_name]
    df = pd.DataFrame(list(col.find({})))
    return df

def get_prediction():
    
    #### Load From DataBase 
    df = get_mongo_data('PrivateDB','productiondata').sort_values(by='TimeStamp')
    test_data = df[['WORKDAETIME_PRE','CYCLE_TIME_PRE','CYCLE_TIME_CURR','INJ_TIME_CURR','CHANGING_PRES','MAX_INJ_PRES',
          'AVG_HOLDING_PRES2','HOPPER_MAX_TEMP','MOLD_TEMP2','SHAPE_PRES_CURR']]
    
    #### Load Saved ML & Predict 
    model = joblib.load('ml.pkl') # lightgbm balanced weight 
    pred = model.predict(test_data)
    pred_df = pd.DataFrame(pred).rename(columns={0:'Predictions'})
    df_addpred = pd.concat([df,pred_df],axis=1).drop(['passorfail','prediction'],axis=1)
    df_addpred['PassOrFail'] = df_addpred['Predictions'].replace({0:'Pass', 1:'Fail'})
    
    #### Predict One Recent Value of Real Time inputs 
    if pred.tolist()[-1] == 0.0: 
        print("prediction_result:",pred.tolist()[-1]," ==> Pass")
    else:
        print("prediction_result:",pred.tolist()[-1]," ==> Fault")
        
    return df_addpred
    
df_addpred = get_prediction()

prediction_result: 0.0  ==> Pass
